# [01] Cross Entropy in Cartpole 

### Imports & Constants

In [19]:
import os
import datetime
import numpy as np
from collections import namedtuple

import gym

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

<br> 

### Neural Net

In [2]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

<br> 

### Helper Functions

In [62]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def sample_episodes(env, network, batch_size):
    '''Retrives a batch of episodes ='''
    
    episodes = [] 
    episode_steps = [] 
    episode_reward = 0.0
    softmax = nn.Softmax(dim=1)
    observation = env.reset()
    
    while True:
        
        # Convert observation into tensor 
        observation_t = torch.FloatTensor([observation])
        
        # Get action via NN and random sampling 
        action_probs_t = softmax(network(observation_t))
        action_probs = action_probs_t.data.numpy()[0]
        action = np.random.choice(len(action_probs), p=action_probs)
        
        # Take step 
        next_observation, reward, is_done, _ = env.step(action)
        
        # Accumulate reward 
        episode_reward += reward
        
        # Save step 
        step = EpisodeStep(observation=observation, action=action)
        episode_steps.append(step)
        
        if is_done:
            
            # Save episode 
            episode = Episode(reward=episode_reward, steps=episode_steps)
            episodes.append(episode)
            
            # Reset 
            episode_reward = 0.0
            episode_steps = []
            next_observation = env.reset()
            
            if len(episodes) == batch_size:
                
                # Return episodes when we've accumulated enough  
                yield episodes
                episodes = []
        
        observation = next_observation


def filter_episodes(episodes, percentile):
    '''Extracts the top/elite episodes'''
    
    # Extract rewards from episodes 
    rewards = list(map(lambda s: s.reward, episodes))
    
    # Calculate reward boundary & mean
    reward_boundary = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    # Save the "elite" episodes 
    train_observations = []
    train_actions = []
    for reward, steps in episodes:
        
        if reward < reward_boundary:
            continue
        
        train_observations.extend(map(lambda step: step.observation, steps))
        train_actions.extend(map(lambda step: step.action, steps))

    # Convert to tensors 
    train_observations_t = torch.FloatTensor(train_observations)
    train_actions_t = torch.LongTensor(train_actions)
    
    return train_observations_t, \
           train_actions_t, \
           reward_boundary, \
           reward_mean

<br> 

### Main 

In [77]:
# Create environment 
env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="monitor", force=True)

# Observation & action space size 
observation_size = env.observation_space.shape[0]
num_actions = env.action_space.n

# Network, Loss, Optimizer 
network = Net(observation_size, HIDDEN_SIZE, num_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=network.parameters(), lr=0.01)

# Log directory for TensorBoard
log_dir = os.path.join('./01_runs', datetime.datetime.now().strftime("%Y:%m:%d-%H%M%S"))
writer = SummaryWriter(log_dir)
print('(Saving to ' + log_dir + ')\n')

# Training 
for epoch, batch in enumerate(sample_episodes(env, network, BATCH_SIZE)):
        
    observations_t, actions_t, reward_boundary, reward_mean = filter_episodes(batch, PERCENTILE)
        
    optimizer.zero_grad()
    action_scores_t = network(observations_t)
    loss_t = objective(action_scores_t, actions_t)
    loss_t.backward()
    optimizer.step()
    
    print("%d: loss=%.3f, reward_mean=%.1f, reward_boundary=%.1f" % (epoch, loss_t.item(), reward_mean, reward_boundary))
    writer.add_scalar("Loss", loss_t.item(), epoch)
    writer.add_scalar("Reward Boundary", reward_boundary, epoch)
    writer.add_scalar("Reward Mean", reward_mean, epoch)
        
    if reward_mean > 199:
        print("Solved!")
        break
    
writer.close()

(Saving to ./01_runs/2020:09:17-123443)

0: loss=0.686, reward_mean=23.0, reward_boundary=28.0
1: loss=0.688, reward_mean=25.3, reward_boundary=30.0
2: loss=0.682, reward_mean=27.2, reward_boundary=27.0
3: loss=0.656, reward_mean=30.0, reward_boundary=31.0
4: loss=0.660, reward_mean=39.8, reward_boundary=44.0
5: loss=0.639, reward_mean=51.4, reward_boundary=64.0
6: loss=0.627, reward_mean=63.1, reward_boundary=86.5
7: loss=0.624, reward_mean=50.1, reward_boundary=66.5
8: loss=0.630, reward_mean=66.9, reward_boundary=93.0
9: loss=0.598, reward_mean=76.2, reward_boundary=92.5
10: loss=0.597, reward_mean=80.9, reward_boundary=104.0
11: loss=0.599, reward_mean=80.8, reward_boundary=115.0
12: loss=0.591, reward_mean=92.1, reward_boundary=118.5
13: loss=0.586, reward_mean=92.0, reward_boundary=116.5
14: loss=0.575, reward_mean=84.2, reward_boundary=91.0
15: loss=0.560, reward_mean=103.4, reward_boundary=121.0
16: loss=0.586, reward_mean=114.0, reward_boundary=130.0
17: loss=0.571, reward_mea

<br> 

## TensorBoard

In [ ]:
# Upload all experiments 
!tensorboard dev upload --logdir='./01_runs' --name='01_cross_entropy_cartpole'

In [ ]:
# List all experiments 
!tensorboard dev list

<br>